# Задание 1

In [27]:
from datetime import datetime

In [70]:
the_moscow_times_date = 'Wednesday, October 2, 2002'
the_moscow_times_datetime = datetime.strptime(the_moscow_times_date, '%A, %B %d, %Y')
print('The Moscow Times -', the_moscow_times_datetime)

The Moscow Times - 2002-10-02 00:00:00


In [71]:
the_guardian_date = 'Friday, 11.10.13'
the_guardian_datetime = datetime.strptime(the_guardian_date, '%A, %d.%m.%y')
print('The Guardian -', the_guardian_datetime)

The Guardian - 2013-10-11 00:00:00


In [72]:
daily_news_date = 'Thursday, 18 August 1977'
daily_news_datetime = datetime.strptime(daily_news_date, '%A, %d %B %Y')
print('Daily News -', daily_news_datetime)

Daily News - 1977-08-18 00:00:00


# Задание 2

In [66]:
from datetime import timedelta

def date_range(start_date_str, end_date_str):
    try:
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    except:
        return []

    if(end_date < start_date):
        return []

    current_dt = start_date
    result = []
    while current_dt <= end_date:
        result.append(current_dt.strftime('%Y-%m-%d'))

        current_dt += timedelta(days=1)

    return result

In [67]:
# проверка 1
date_range('2022-01-01', '2022-01-03')

['2022-01-01', '2022-01-02', '2022-01-03']

In [68]:
# проверка 2
date_range('2022-01-03', '2022-01-01')

[]

In [69]:
# проверка 3
date_range('2022-02-30', '2022-02-31')

[]